In [1]:
import numpy as np
import pandas as pd
import time
from model import Model

# ダウンロードしてきたやつ
INDEIES = ["N225",
           "AORD",
           "HSI",
           "GDAXI",
           "FTSE",
           "NYA",
           "DJI",
           "GSPC",
          ]
ANSWER = "N225"

def getClosing():
    closing = pd.DataFrame()
    for index in INDEIES:
        # na_valuesは文字列"null"のとき空として扱う
        df = pd.read_csv("./data/input1/" + index + ".csv", na_values=["null"])
        df["Date"] = pd.to_datetime(df["Date"])
        df = df.set_index("Date")
        closing[index] = df["Close"]
    # 空の部分は古いので埋める
    closing = closing.fillna(method="ffill")
    for index in INDEIES:
        #closing[index] = closing[index] / max(closing[index])
        closing[index] = np.log(closing[index] / closing[index].shift())
    closing["negative"] = 0
    # closing[ANSWER] < 0 の行のnegativeに 1
    closing.loc[closing[ANSWER] < 0, "negative"] = 1
    closing["positive"] = 0
    # closing[ANSWER] >= 0 の行のpositiveに 1
    closing.loc[closing[ANSWER] >= 0, "positive"] = 1
    return closing

def getTraningData():
    closing = getClosing()
    # 1~3日前のデータを予測に使う
    days_before = range(1,4)
    answers = pd.DataFrame(columns = ["positive", "negative"])
    columns = []
    for index in INDEIES:
        columns += [index + "_" + str(before) for before in days_before]
    features = pd.DataFrame(columns = columns)
    # なんで7から？
    for i in range(7, len(closing)):
        # 予測の部分は当日のデータ
        answers = answers.append({
            "positive" : closing["positive"].iloc[i],
            "negative" : closing["negative"].iloc[i]}, ignore_index=True)
        data={}
        # ほかの指標は１個前のデータ
        for index in INDEIES:
            for before in days_before :
                data[index + "_" + str(before)] = closing[index].iloc[i - before]
        features = features.append(data, ignore_index=True)
    # 予測する元のデータ , 予測するべきデータ
    return features,answers

if __name__ == "__main__":
    start = time.time()
    features,answers = getTraningData()
    for test in range(4,5):
        layers = [50,25]
        model = Model(features,answers,layers,test*0.2,test*0.2+0.2)
        model.train()
        model.test()
    print("elapsed_time :", time.time()-start, "[sec]")

3000 0.680328
6000 0.684939
9000 0.684426
12000 0.685963
15000 0.686988
18000 0.688525
21000 0.688525
24000 0.688525
27000 0.690574
30000 0.689037
Up_Accuracy	= 0.652985
Down_Accuracy	= 0.604545
Accuracy	= 0.631148
elapsed_time : 110.14158463478088 [sec]
